In [2]:
# !pwd

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import re
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('data_clean.csv')
data.head()

,Unnamed: 0,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,...,Epi_Link_Exposure,disease_len_days,geo_address,address,Travel History Binary,region_inc,average income (in thousand pesos),city,region,Population Density
0,0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,['wife of ph2'],2020-01-21 00:00:00-05:00,...,1,53 days 23:00:00.000000000,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...","San Lazaro Hospital, Quiricada Street, Baranga...",1,NCR,460,Manila,Metro Manila,"71,263"
1,1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,['husband of ph1'],2020-01-18 00:00:00-05:00,...,1,14 days 00:00:00.000000000,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...","San Lazaro Hospital, Quiricada Street, Baranga...",1,NCR,460,Manila,Metro Manila,"71,263"
2,2,PH00003,F,60,chinese,None,Yes (China),china,traveled to cebu and bohol. was already in chi...,2020-01-21 00:00:00-05:00,...,0,53 days 23:00:00.000000000,"{'public_building': 'Francisco Duque Hall', 'r...","Francisco Duque Hall, Tayuman Street, Santa Cr...",1,NCR,460,Manila,Metro Manila,"71,263"
3,3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,['NaN'],2020-03-03 00:00:00-05:00,...,1,15 days 23:00:00.000000000,{'hospital': 'University of the East - Ramon M...,University of the East - Ramon Magsaysay Memor...,1,NCR,460,Galas,Metro Manila,20785
4,4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,['husband of ph6'],2020-02-25 00:00:00-05:00,...,1,15 days 23:00:00.000000000,{'hospital': 'Research Institute for Tropical ...,"Research Institute for Tropical Medicine, Corp...",0,NCR,460,Muntinlupa,Metro Manila,"12,692"


In [4]:
data.shape

(3016, 119)

In [6]:
data.columns.tolist()

['Unnamed: 0',
 'phcaseno',
 'Sex',
 'Age',
 'Nationality',
 'Residence in the Philippines',
 'History of Travel / Exposure',
 'Travel History',
 'Epi_Link',
 'Date of Onset of Symptoms',
 'Date of Admission',
 'Date of Lab Confirmation',
 'Date of Announcement to the Public',
 'Admission / Consultation',
 'Symptoms',
 'other_disease',
 'Status',
 'Health Status',
 'Date of Final Status (recovered/expired)',
 'Final Diagnosis',
 'Age Group',
 'Location',
 'Location 1',
 'Location 2',
 'Latitude',
 'Longitude',
 'Residence Lat',
 'Residence Long',
 'symp_ body malaise',
 'symp_ colds',
 'symp_ comorbidity',
 'symp_ cough',
 'symp_ difficulty in breathing',
 'symp_ dizziness',
 'symp_ dysphagia',
 'symp_ headache',
 'symp_ shortness of breath',
 'symp_ sore throat',
 'symp_asymptomatic',
 'symp_cough',
 'symp_difficulty of breathing',
 'symp_dyspnea',
 'symp_fever',
 'symp_sore throat',
 'dis_  diabetes',
 'dis_  renal disease',
 'dis_ acute kidney injury',
 'dis_ asthma',
 'dis_ atheros

In [7]:
# target variable information (after clustering)
data.groupby('Health Status').size()

Health Status
asymptomatic        38
critical             3
died               108
for validation    2754
mild                57
recovered           50
severe               6
dtype: int64

In [40]:
# from master

# Note: upload this file at run-time


# remove for validation
data_clean = data[data['Health Status'] != "for validation"]
data_clean=data_clean.reset_index(drop=True)
data_clean.columns

Index(['Unnamed: 0', 'phcaseno', 'Sex', 'Age', 'Nationality',
       'Residence in the Philippines', 'History of Travel / Exposure',
       'Travel History', 'Epi_Link', 'Date of Onset of Symptoms',
       ...
       'Epi_Link_Exposure', 'disease_len_days', 'geo_address', 'address',
       'Travel History Binary', 'region_inc',
       'average income (in thousand pesos)', 'city', 'region',
       'Population Density'],
      dtype='object', length=119)

In [41]:
data_clean = data_clean.iloc[:, 1:]
data_clean.head()

,phcaseno,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Epi_Link_Exposure,disease_len_days,geo_address,address,Travel History Binary,region_inc,average income (in thousand pesos),city,region,Population Density
0,PH00001,F,38,chinese,None,Yes (China); Wife of PH02,china,['wife of ph2'],2020-01-21 00:00:00-05:00,Sat Jan 25 00:00:00 EST 2020,...,1,53 days 23:00:00.000000000,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...","San Lazaro Hospital, Quiricada Street, Baranga...",1,NCR,460,Manila,Metro Manila,"71,263"
1,PH00002,M,44,chinese,None,Yes (China); Husband of PH1,china,['husband of ph1'],2020-01-18 00:00:00-05:00,Sat Jan 25 00:00:00 EST 2020,...,1,14 days 00:00:00.000000000,"{'hospital': 'San Lazaro Hospital', 'road': 'Q...","San Lazaro Hospital, Quiricada Street, Baranga...",1,NCR,460,Manila,Metro Manila,"71,263"
2,PH00003,F,60,chinese,None,Yes (China),china,traveled to cebu and bohol. was already in chi...,2020-01-21 00:00:00-05:00,Wed Jan 22 00:00:00 EST 2020,...,0,53 days 23:00:00.000000000,"{'public_building': 'Francisco Duque Hall', 'r...","Francisco Duque Hall, Tayuman Street, Santa Cr...",1,NCR,460,Manila,Metro Manila,"71,263"
3,PH00004,M,48,filipino,Taguig City,Yes (Japan),japan,['NaN'],2020-03-03 00:00:00-05:00,Fri Mar 06 00:00:00 EST 2020,...,1,15 days 23:00:00.000000000,{'hospital': 'University of the East - Ramon M...,University of the East - Ramon Magsaysay Memor...,1,NCR,460,Galas,Metro Manila,20785
4,PH00005,M,62,filipino,"Cainta, Rizal",Husband of PH06,none,['husband of ph6'],2020-02-25 00:00:00-05:00,Thu Mar 05 00:00:00 EST 2020,...,1,15 days 23:00:00.000000000,{'hospital': 'Research Institute for Tropical ...,"Research Institute for Tropical Medicine, Corp...",0,NCR,460,Muntinlupa,Metro Manila,"12,692"


In [42]:
data.groupby('Health Status').size()

Health Status
asymptomatic        38
critical             3
died               108
for validation    2754
mild                57
recovered           50
severe               6
dtype: int64

In [43]:
print(data_clean['Nationality'].value_counts() )

def nationality_recode(series):
    if series != 'filipino':
        return 'other'
    else: 
        return 'filipino'

data_clean['Nationality'] = data_clean['Nationality'].apply(nationality_recode)
data_clean['Nationality'] = pd.get_dummies(data_clean['Nationality'])
data_clean['Nationality'].value_counts()

filipino              241
for validation         10
chinese                 3
american                3
thai                    2
british, uk             2
chinese, taiwanese      1
Name: Nationality, dtype: int64


1    241
0     21
Name: Nationality, dtype: int64

In [44]:
data_clean['disease_len_days']

0      53 days 23:00:00.000000000
1      14 days 00:00:00.000000000
2      53 days 23:00:00.000000000
3      15 days 23:00:00.000000000
4      15 days 23:00:00.000000000
                  ...            
257    19 days 23:00:00.000000000
258                           NaN
259     0 days 00:00:00.000000000
260     0 days 00:00:00.000000000
261     0 days 00:00:00.000000000
Name: disease_len_days, Length: 262, dtype: object

In [45]:
data_clean['disease_len_days'] = data_clean["disease_len_days"].str.split(" ", 1, expand=True)[0]

In [46]:
data_clean['disease_len_days']

0       53
1       14
2       53
3       15
4       15
      ... 
257     19
258    NaN
259      0
260      0
261      0
Name: disease_len_days, Length: 262, dtype: object

In [47]:
import numpy as np
#np.array(data_clean['Health Status'])

def status_to_numeric(x):
    if x=='died':
        return 5
    if x=='critical':
        return 4
    if x=='severe':
        return 3
    if x=='mild':
        return 2
    if x=='asymptomatic':
        return 1
    if x=='recovered':
        return 0

In [48]:

data_clean['HealthStatusNUM'] = data_clean['Health Status'].apply(status_to_numeric)

In [49]:
data_clean['HealthStatusNUM']

0      0
1      5
2      0
3      0
4      5
      ..
257    0
258    2
259    5
260    5
261    5
Name: HealthStatusNUM, Length: 262, dtype: int64

In [51]:

# data_clean = data_clean.drop(['Health Status', 'HealthStatusNUM', 'phcaseno','Residence in the Philippines', 'Date of Onset of Symptoms', 'Date of Admission', 'Date of Lab Confirmation', 'Date of Announcement to the Public', 'Admission / Consultation', 'Symptoms', 'other_disease', 'Status', 'Date of Final Status (recovered/expired)', 'Final Diagnosis', 'Age Group', 'Location', 'Location 1', 'Location 2', 'Residence Lat', 'Residence Long', 'geo_address'], axis=1)

data_clean.columns.tolist()

['phcaseno',
 'Sex',
 'Age',
 'Nationality',
 'Residence in the Philippines',
 'History of Travel / Exposure',
 'Travel History',
 'Epi_Link',
 'Date of Onset of Symptoms',
 'Date of Admission',
 'Date of Lab Confirmation',
 'Date of Announcement to the Public',
 'Admission / Consultation',
 'Symptoms',
 'other_disease',
 'Status',
 'Health Status',
 'Date of Final Status (recovered/expired)',
 'Final Diagnosis',
 'Age Group',
 'Location',
 'Location 1',
 'Location 2',
 'Latitude',
 'Longitude',
 'Residence Lat',
 'Residence Long',
 'symp_ body malaise',
 'symp_ colds',
 'symp_ comorbidity',
 'symp_ cough',
 'symp_ difficulty in breathing',
 'symp_ dizziness',
 'symp_ dysphagia',
 'symp_ headache',
 'symp_ shortness of breath',
 'symp_ sore throat',
 'symp_asymptomatic',
 'symp_cough',
 'symp_difficulty of breathing',
 'symp_dyspnea',
 'symp_fever',
 'symp_sore throat',
 'dis_  diabetes',
 'dis_  renal disease',
 'dis_ acute kidney injury',
 'dis_ asthma',
 'dis_ atherosclerosis',
 'di

In [52]:
data['Age Group']

0       31 - 40
1       41 - 50
2       51 - 60
3       41 - 50
4       61 - 70
         ...   
3011    51 - 60
3012    21 - 30
3013    41 - 50
3014    31 - 40
3015    21 - 30
Name: Age Group, Length: 3016, dtype: object

In [ ]:
# do not drop age group and other important vars
]:
data_clean = data_clean.drop(['Health Status', 'HealthStatusNUM', 'phcaseno','Residence in the Philippines', 'Date of Onset of Symptoms', 'Date of Admission', 'Date of Lab Confirmation', 'Date of Announcement to the Public', 'Admission / Consultation', 'Symptoms', 'other_disease', 'Status', 'Date of Final Status (recovered/expired)', 'Final Diagnosis', 'Age Group', 'Location', 'Location 1', 'Location 2', 'Residence Lat', 'Residence Long', 'geo_address'], axis=1)

In [39]:
data_clean.columns

Index(['Sex', 'Age', 'Nationality', 'History of Travel / Exposure',
       'Travel History', 'Epi_Link', 'Latitude', 'Longitude',
       'symp_ body malaise', 'symp_ colds', 'symp_ comorbidity', 'symp_ cough',
       'symp_ difficulty in breathing', 'symp_ dizziness', 'symp_ dysphagia',
       'symp_ headache', 'symp_ shortness of breath', 'symp_ sore throat',
       'symp_asymptomatic', 'symp_cough', 'symp_difficulty of breathing',
       'symp_dyspnea', 'symp_fever', 'symp_sore throat', 'dis_  diabetes',
       'dis_  renal disease', 'dis_ acute kidney injury', 'dis_ asthma',
       'dis_ atherosclerosis', 'dis_ atherosclerotic cardiovascular disease',
       'dis_ benign prostatic hyperplasia',
       'dis_ benign prostatic hypertrophy', 'dis_ bronchial asthma',
       'dis_ cardiac disease', 'dis_ cardiac dysrhythmia',
       'dis_ cardiomyopathy', 'dis_ cardiovascular disease',
       'dis_ cerebrovascular accident infarc',
       'dis_ cerebrovascular disease infarction',
       